In [640]:
import pandas as pd
import datetime as dt
import sqlite3
import numpy as np
%run C:\Users\sgasmi\Notebooks\Column_autosize.ipynb

In [641]:
kpp = pd.read_csv(r'C:\Users\sgasmi\Desktop\Extractions\KPP.csv', sep="|")
fafr = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Effectif_Data_Quality.xlsx", converters={'Matricule':str})
eff_app = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Employee.xlsx")
grad_ref = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Grading_03052021.xlsx", converters={'Grade':str}) # Fichier Thi Hien

## Préparation de la base effectif

In [642]:
res = fafr[["Matricule", "Nom", "Prénom", "Etablissement"]]
res = res.drop_duplicates()

## Préparation du fichier KPP

In [643]:
kpp.loc[kpp["enddate"].isna()==True, "enddate"] = pd.to_datetime("01/01/2050 00:00:00", format="%d/%m/%Y %H:%M:%S").strftime("%d/%m/%Y %H:%M:%S") # On remplace les dates de fin des lignes actives
kpp = kpp.merge(eff_app[["username", "extra_local_id_payroll_fra","extra_home_contract_id"]], on="username", how="left")
kpp["extra_local_id_payroll_fra"] = kpp.apply(lambda x: x["extra_local_id_payroll_fra"] if pd.isna(x["extra_local_id_payroll_fra"]) == False else x["extra_home_contract_id"], axis=1)
kpp["Today"] = dt.date.today()
col = [x.replace(" ", "_") for x in kpp.columns]
kpp.colums = col

<ipython-input-643-207c40eb8c06>:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  kpp.colums = col


In [644]:
kpp["enddate"] = kpp["enddate"].apply(lambda x :str(x).replace("9999", "2050"))

In [645]:
kpp["startdate"] = pd.to_datetime(kpp["startdate"],format='%d/%m/%Y %H:%M:%S').dt.date
print("ok")
kpp["enddate"] = pd.to_datetime(kpp["enddate"],format='%d/%m/%Y %H:%M:%S').dt.date

ok


In [646]:
kpp = kpp.rename({"extra_local_id_payroll_fra":"Payroll ID"}, axis=1)
kpp.columns

Index(['username', 'referential', 'level1', 'level2', 'level3', 'startdate',
       'enddate', 'comment', 'Payroll ID', 'extra_home_contract_id', 'Today'],
      dtype='object')

In [647]:
#Création d'une base de donnée in memory
conn = sqlite3.connect(':memory:')
# Ajout des Dataframe dans la base de données
kpp.to_sql('kpp', conn, index=False)
# Création d'une requête sql
req = "select * from kpp where Today between startdate and enddate"
# Exécution de la requête et stokage du résultat dans un DataFrame
kpp = pd.read_sql_query(req, conn)
#kpp = kpp.drop("Today", axis=1)
kpp= kpp.loc[kpp["Payroll ID"].isna()==False].copy()
kpp= kpp.reset_index(drop=True)
kpp["Payroll ID"] = kpp["Payroll ID"].apply(lambda x: x[4:])
fafr["Matricule"] = fafr["Matricule"].astype("str")

C:\Users\sgasmi\Miniconda3\lib\site-packages\pandas\core\generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [648]:
#resize_column([kpp], "Ecart sal", r"C:\Users\sgasmi\Desktop\Export Python\kpp_test.xlsx")

In [649]:
kpp_grade = kpp.loc[kpp["referential"]=="Grade"].reset_index(drop=True)

In [650]:
fafr = fafr.merge(kpp_grade, right_on="Payroll ID", left_on="Matricule", how="left")

#Filtre sur les cadres

In [651]:
fafr = fafr.loc[fafr["Catég conv"].isin(["C", "CS", "D"])].reset_index(drop=True)

In [652]:
grade_compl = len(set(fafr.loc[fafr["referential"]=="Grade", "Matricule"]))

In [653]:
total = len(fafr["Matricule"].unique())

In [654]:
doublon = fafr.groupby(["Matricule"])["Matricule"].count().to_frame(name = 'Nb salarié(s)').reset_index()

In [655]:
nb_doublon = len(set(doublon.loc[doublon["Nb salarié(s)"]>1, "Matricule"]))
ratio_comp = round((grade_compl/total)*100,2)
nb_vide = len(set(fafr.loc[fafr["referential"].isna()==True, "Matricule"]))

In [656]:
print("Le taux complétude des données de grading dans MyHR est {0} %. ".format(ratio_comp,total))
print("En effet, le nombre de salariés ayant un grading complété est de {0} pour {1} salariés au total (population cadre)".format(grade_compl, total ))
print("{0} salariés ont plusieurs occurences actives de grading (anomalies?)".format(nb_doublon))

Le taux complétude des données de grading dans MyHR est 92.01 %. 
En effet, le nombre de salariés ayant un grading complété est de 4309 pour 4683 salariés au total (population cadre)
269 salariés ont plusieurs occurences actives de grading (anomalies?)


In [657]:
Grading_vide= fafr.loc[fafr["referential"].isna()==True].copy()

In [658]:
today = dt.date.today()
Grading_vide["Date"] = today
today = today.strftime('%d %m %Y')
today = today.replace(" ", "")

In [659]:
Grading_vide["Anomalie"] = "Code Grading manquant"

Grading_vide= Grading_vide[["Matricule", "Nom", "Prénom", "BU", "Etablissement", "Niveau", "Anomalie","level2", "Ancienneté groupe","Date"]]

## Comparaison des grading existants dans MyHR avec ceux présents dans le fichier grading référentiel

### Préparation du fichier grading référentiel

In [660]:
grad_ref = grad_ref.merge(eff_app[["employeenumber","extra_local_id_payroll_fra"]], left_on="MyHR ID", right_on="employeenumber", how="left")
grad_ref = grad_ref.merge(eff_app[["employeenumber","extra_home_contract_id"]], left_on="MyHR ID", right_on="employeenumber", how="left")
grad_ref["extra_local_id_payroll_fra"] = grad_ref.apply(lambda x: x["extra_local_id_payroll_fra"] if pd.isna(x["extra_local_id_payroll_fra"]) == False else x["extra_home_contract_id"], axis=1)

In [661]:
grad_ref= grad_ref.rename({"extra_local_id_payroll_fra":"Payroll ID"}, axis=1)

In [662]:
grad_ref["Payroll ID"] = grad_ref["Payroll ID"].apply(lambda x: str(x)[4:])

In [663]:
fafr = fafr.merge(grad_ref[["Payroll ID", "Grade"]], left_on="Matricule", right_on="Payroll ID", how="left")

In [664]:
fafr.loc[fafr["Matricule"]=="35965","level2"]

756    NaN
Name: level2, dtype: object

In [665]:
fafr["Ecart_grade"] = fafr.apply(lambda x: "Ecart" if str(x["level2"]) != str(x["Grade"]) else np.nan, axis=1)

In [666]:
ecart = fafr.loc[fafr["Ecart_grade"]=="Ecart"]
ecart["Anomalie"] =  "Ecart fichier de référence grading"
ecart["Date"] = dt.date.today()
ecart = ecart[["Matricule", "Nom", "Prénom", "BU", "Etablissement", "Niveau", "Anomalie","level2", "Ancienneté groupe","Date"]].copy()

<ipython-input-666-d7836b4f0fae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecart["Anomalie"] =  "Ecart fichier de référence grading"
<ipython-input-666-d7836b4f0fae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecart["Date"] = dt.date.today()


In [667]:
Anomalie = Grading_vide
Anomalies = Anomalie.append(ecart)

In [668]:
resize_column([ecart], "Check Grading", r"C:\Users\sgasmi\Desktop\Export Python\Ecart.xlsx")

In [669]:
resize_column([Anomalies], "Check Grading", r"C:\Users\sgasmi\Desktop\Power BI applications\Sources MyHR\Anomalie_MyHR_" + str(today) + ".xlsx")